# Descomposición en Valores Singulares (SVD) y Reconstrucción de Imágenes con Julia

Este notebook aborda cómo usar la **Descomposición en Valores Singulares (SVD)** para procesar imágenes en Julia. A través de este proceso, se pueden reconstruir imágenes con diferentes grados de precisión. El código incluye varias funciones, cada una con un propósito específico.

---

## Importación de librerías necesarias

Se requiere cargar las siguientes librerías para procesar imágenes y trabajar con matrices.

In [1]:
using Images, LinearAlgebra

La función `FindSVD` calcula la descomposición en valores singulares (SVD) de una matriz \( M \). 
## Descomposición SVD

La descomposición SVD descompone una matriz $ M $ en tres componentes principales:

$
M = U \Sigma V^T
$

Donde:

- $ U $ es una matriz ortogonal que contiene los vectores singulares de la izquierda.
- $ \Sigma $ es una matriz diagonal que contiene los valores singulares.
- $ V^T $ es la traspuesta de una matriz ortogonal \( V \), que contiene los vectores singulares de la derecha.

---

## Información sobre la estructura `SVD`

El parámetro $ F $ contiene las componentes $ U $, $ \Sigma $, $ V $, y $ V^T $ como parte de la estructura `SVD`. Esta función imprime:

- Los **tipos de datos**.
- Las **dimensiones** de cada componente.

In [2]:
function FindSVD( M::Matrix )
	F = svd( M )
	return F
end
 
function PrintSVDInfo( F::SVD )
	println("PrintSVDInfo")
	println("F.U  Info: Type = ", typeof(F.U),  "  size=", size(F.U))
	println("F.S  Info: Type = ", typeof(F.S),  "  size=", size(F.S))
	println("F.V  Info: Type = ", typeof(F.V),  "  size=", size(F.V))
	println("F.Vt Info: Type = ", typeof(F.Vt), "  size=", size(F.Vt))
end

PrintSVDInfo (generic function with 1 method)

## Función `FindApproxImg`

### Propósito

Esta función utiliza las componentes de la descomposición SVD para reconstruir aproximaciones de la matriz original, reduciendo el rango de valores singulares considerados.

---

### Detalles de la implementación

1. Selección de $ k $:  
   La reconstrucción se realiza considerando $ k = 10 $, $ k = 50 $, $ k = 100 $, y $ k = 400 $, lo que permite observar cómo la calidad de la reconstrucción mejora con un mayor número de valores singulares.

2. Reconstrucción matemática:  
   Para cada valor de $ k $, la matriz aproximada $ M_k $ se calcula como:

   $$
   M_k = U[:, 1:k] \cdot \Sigma_k \cdot V[:, 1:k]^T
   $$

   Donde:
   - $ U[:, 1:k] $ selecciona las primeras $ k $ columnas de la matriz $ U $.
   - $ \Sigma_k $ es una matriz diagonal que contiene los $ k $ valores singulares más grandes.
   - $ V[:, 1:k] $ selecciona las primeras $ k $ columnas de la matriz $ V $.

3. Escalado:  
   Los valores de $ M_k $ se escalan con un factor de $ 0.99 $ y se transforman a valores absolutos para garantizar que se mantengan en el rango válido para imágenes.

4. Guardado de las aproximaciones:  
   Cada aproximación $ M_k $ se guarda como una imagen con nombres como `Aprox10.jpg`, `Aprox50.jpg`, etc.

---

In [12]:
function FindApproxImg(F::SVD)
	k = 10
	M = F.U[:, 1:k] * Diagonal(F.S[1:k]) * F.V[:, 1:k]'
	M = abs.( M .* 0.99)
	save("Aprox10.jpg", M)
    println("Imagen con k=",k," creada")
 
	k = 50
	M = F.U[:, 1:k] * Diagonal(F.S[1:k]) * F.V[:, 1:k]'
	M = abs.( M .* 0.99)
	save("Aprox50.jpg", M)
    println("Imagen con k=",k," creada")
 
	k = 100
	M = F.U[:, 1:k] * Diagonal(F.S[1:k]) * F.V[:, 1:k]'
	M = abs.( M .* 0.99)
	save("Aprox100.jpg", M)
    println("Imagen con k=",k," creada")

	k = 400
	M = F.U[:, 1:k] * Diagonal(F.S[1:k]) * F.V[:, 1:k]'
	M = abs.( M .* 0.99)
	save("Aprox400.jpg", M)
    println("Imagen con k=",k," creada")
end

FindApproxImg (generic function with 1 method)


## Función `CreateArrayFromImgFile`

### Propósito

Convierte un archivo de imagen en una matriz para su procesamiento.


In [14]:
File = "TestImage.jpg"
TestImage = load(File)
TestImageGray = Gray.(TestImage)
TestImageGrayMatrix = convert( Matrix{Float64}, TestImageGray)
F = FindSVD( TestImageGrayMatrix )
PrintSVDInfo(F)
println()
FindApproxImg(F)

PrintSVDInfo
F.U  Info: Type = Matrix{Float64}  size=(2000, 1500)
F.S  Info: Type = Vector{Float64}  size=(1500,)
F.V  Info: Type = Adjoint{Float64, Matrix{Float64}}  size=(1500, 1500)
F.Vt Info: Type = Matrix{Float64}  size=(1500, 1500)

Imagen con k=10 creada
Imagen con k=50 creada
Imagen con k=100 creada
Imagen con k=400 creada
